In [3]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, pi
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score, mean_squared_error

# Double Pendulum Parameters
g = 9.81
L1 = 1
L2 = 1
m1 = 1.0
m2 = 1.0

# Function for Double Pendulum System
def f(r, t, L1, L2):
    theta1 = r[0]
    omega1 = r[1]
    theta2 = r[2]
    omega2 = r[3]

    ftheta1 = omega1
    fomega1 = (-g * (2 * m1 + m2) * sin(theta1) - m2 * g * sin(theta1 - 2 * theta2) - 2 * sin(theta1 - theta2) * m2 *
               (omega2**2 * L2 + omega1**2 * L1 * cos(theta1 - theta2))) / (L1 * (2 * m1 + m2 - m2 * cos(2 * theta1 - 2 * theta2)))

    ftheta2 = omega2
    fomega2 = (2 * sin(theta1 - theta2) * (omega1**2 * L1 * (m1 + m2) + g * (m1 + m2) * cos(theta1) + omega2**2 * L2 * m2 *
                                           cos(theta1 - theta2))) / (L2 * (2 * m1 + m2 - m2 * cos(2 * theta1 - 2 * theta2)))

    return np.array([ftheta1, fomega1, ftheta2, fomega2], float)

# Simulation Parameters
a = 0.0
b = 10
N = 2000
h = (b - a) / N

angles = [[120, 0], [121, 1], [120, 2], [120, 4], [120, 6], [120, 8], [120, 10],
          [122, 0], [122, 2], [122, 4], [122, 6], [122, 8], [122, 10],
          [124, 0], [124, 2], [124, 4], [124, 6], [124, 8], [124, 10],
          [126, 0], [126, 2], [126, 4], [126, 6], [126, 8], [126, 10],
          [128, 0], [128, 2], [128, 4], [128, 6], [128, 8], [128, 10],
          [130, 0], [130, 2], [130, 4], [130, 6], [130, 8], [130, 10]]

# Simulate the double pendulum and save the data
for x in angles:
    tpoints = np.arange(a, b, h)
    theta1_points = np.zeros_like(tpoints)
    theta2_points = np.zeros_like(tpoints)

    q = np.array([x[0] * pi / 180, 0, x[1] * pi / 180, 0], float)

    for i, t in enumerate(tpoints):
        theta1_points[i] = q[0] * 180 / pi
        theta2_points[i] = q[2] * 180 / pi

        k1 = h * f(q, t, L1, L2)
        k2 = h * f(q + 0.5 * k1, t + 0.5 * h, L1, L2)
        k3 = h * f(q + 0.5 * k2, t + 0.5 * h, L1, L2)
        k4 = h * f(q + k3, t + h, L1, L2)
        q += (k1 + 2 * k2 + 2 * k3 + k4) / 6

    data = np.stack((theta1_points, theta2_points), axis=1)
    np.save(f'pendulum_data_{str(x[0])}_{str(x[1])}.npy', data)

# FFNN Dataset Class
class PendulumDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)  # Input features (N, 3)
        self.y = torch.tensor(y, dtype=torch.float32)  # Target labels (N, 2)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Function to create inputs and outputs for the neural network
def create_io(data):
    x, x_1, x_2, y_1, y_2 = [], [], [], [], []
    for starting in data:
        starting_theta_1_degrees = int(starting.split("_")[0])
        starting_theta_2_degrees = int(starting.split("_")[1])

        starting_theta_1 = starting_theta_1_degrees * pi / 180
        starting_theta_2 = starting_theta_2_degrees * pi / 180

        angle_data = data[starting]
        for i in range(len(angle_data)):
            x.append(tpoints[i])
            x_1.append(starting_theta_1)
            x_2.append(starting_theta_2)
            y_1.append(angle_data[i][0])
            y_2.append(angle_data[i][1])
    return x, x_1, x_2, y_1, y_2

# Load and normalize the data
data = {}
for i in angles:
    loaded_data = np.load(f'pendulum_data_{str(i[0])}_{str(i[1])}.npy')
    scaler = MinMaxScaler()
    data_ = scaler.fit_transform(loaded_data)
    data[f'{str(i[0])}_{str(i[1])}'] = data_

x, x_1, x_2, y_1, y_2 = create_io(data)

# Normalize data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

x_combined = np.vstack([x_1, x_2, x]).T
x_scaled = scaler_x.fit_transform(x_combined)

y_combined = np.vstack([y_1, y_2]).T
y_scaled = scaler_y.fit_transform(y_combined)

# Create dataset and dataloaders
dataset = PendulumDataset(x_scaled, y_scaled)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# Define the Feed-Forward Neural Network (FFNN) model class
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = 3
hidden_size = 50
output_size = 2
model = FeedForwardNN(input_size, hidden_size, output_size).to(device)

loss_function = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function for FFNN
def train_model_ffnn(model, train_loader, test_loader, num_epochs):
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            y_pred = model(inputs)
            loss = loss_function(y_pred, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                y_pred = model(inputs)
                val_loss += loss_function(y_pred, targets).item()

        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(test_loader))
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Validation Loss: {val_losses[-1]:.4f}')
    
    return train_losses, val_losses

# Train the model for 100 epochs
train_model_ffnn(model, train_loader, test_loader, 100)

# Evaluate the model
model.eval()
y_preds = []
y_trues = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        y_pred = model(inputs)
        y_preds.append(y_pred.cpu().numpy())
        y_trues.append(targets.cpu().numpy())

# Convert predictions and targets to arrays
y_preds = np.concatenate(y_preds, axis=0)
y_trues = np.concatenate(y_trues, axis=0)

# Rescale predictions back to original scale
y_preds_rescaled = scaler_y.inverse_transform(y_preds)
y_trues_rescaled = scaler_y.inverse_transform(y_trues)

# Calculate R^2 Score
r2 = r2_score(y_trues_rescaled, y_preds_rescaled)
print(f"R^2 Score: {r2:.4f}")


Epoch 1/100, Train Loss: 0.0684, Validation Loss: 0.0573
Epoch 2/100, Train Loss: 0.0514, Validation Loss: 0.0455
Epoch 3/100, Train Loss: 0.0414, Validation Loss: 0.0368
Epoch 4/100, Train Loss: 0.0349, Validation Loss: 0.0322
Epoch 5/100, Train Loss: 0.0318, Validation Loss: 0.0307
Epoch 6/100, Train Loss: 0.0295, Validation Loss: 0.0305
Epoch 7/100, Train Loss: 0.0284, Validation Loss: 0.0277
Epoch 8/100, Train Loss: 0.0274, Validation Loss: 0.0275
Epoch 9/100, Train Loss: 0.0267, Validation Loss: 0.0259
Epoch 10/100, Train Loss: 0.0260, Validation Loss: 0.0252
Epoch 11/100, Train Loss: 0.0254, Validation Loss: 0.0249
Epoch 12/100, Train Loss: 0.0249, Validation Loss: 0.0254
Epoch 13/100, Train Loss: 0.0245, Validation Loss: 0.0245
Epoch 14/100, Train Loss: 0.0241, Validation Loss: 0.0250
Epoch 15/100, Train Loss: 0.0239, Validation Loss: 0.0238
Epoch 16/100, Train Loss: 0.0235, Validation Loss: 0.0233
Epoch 17/100, Train Loss: 0.0233, Validation Loss: 0.0229
Epoch 18/100, Train Los

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from math import sin, cos, pi

from matplotlib import rc
rc('animation', html='jshtml')

def theta_to_xy(theta1, theta2, theta1Len=1, theta2len=1):
    convert = pi/180
    x1 = sin(theta1*convert) * theta1Len
    y1 = -cos(theta1*convert) * theta1Len
    x2 = x1 + sin(theta2*convert) * theta2len
    y2 = y1 - cos(theta2*convert) * theta2len
    return [x1, y1, x2, y2]

x1_points = []
y1_points = []
x2_points = []
y2_points = []

x1_points_p = []
y1_points_p = []
x2_points_p = []
y2_points_p = []

for i in range(len(tpoints)):
    preddy = scaler.inverse_transform(y_pred_90_90)
    actual = scaler.inverse_transform(data["120_0"])

    x1, y1, x2, y2 = theta_to_xy(actual[i][0], actual[i][1])
    x1_points.append(x1)
    y1_points.append(y1)
    x2_points.append(x2)
    y2_points.append(y2)

    t1 = preddy[i][0]
    t2 = preddy[i][1]

    x1p, y1p, x2p, y2p = theta_to_xy(t1, t2)
    x1_points_p.append(x1p)
    y1_points_p.append(y1p)
    x2_points_p.append(x2p)
    y2_points_p.append(y2p)


fig = plt.figure(figsize=(5, 4))
L = 2
ax = fig.add_subplot(autoscale_on=False, xlim=(-L, L), ylim=(-L, L))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], 'r-', lw=1, ms=2, alpha=0.5)
line_p, = ax.plot([], [], 'o-', lw=2)
trace_p, = ax.plot([], [], 'r-', lw=1, ms=2, alpha=0.5)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    trace.set_data([], [])
    line_p.set_data([], [])
    trace_p.set_data([], [])
    time_text.set_text('')
    return line, trace, line_p, trace_p, time_text

trace_p, = ax.plot([], [], 'b-', lw=1, ms=2, alpha=0.5)

def update(i):
    thisx = [0, x1_points[i], x2_points[i]]
    thisy = [0, y1_points[i], y2_points[i]]

    history_x = x2_points[:i]
    history_y = y2_points[:i]

    thisx_p = [0, x1_points_p[i], x2_points_p[i]]
    thisy_p = [0, y1_points_p[i], y2_points_p[i]]

    history_x_p = x2_points_p[:i]
    history_y_p = y2_points_p[:i]

    line.set_data(thisx, thisy)
    trace.set_data(history_x, history_y)
    line_p.set_data(thisx_p, thisy_p)
    trace_p.set_data(history_x_p, history_y_p)
    time_text.set_text(time_template % (i * h))

    if i == len(tpoints) - 1:
        plt.show()

    return line, trace, line_p, trace_p, time_text

ani = FuncAnimation(fig, update, frames=len(tpoints), init_func=init, blit=True, interval=20)
plt.close()
ani
